* `python3.10 -m pip install llama-index-readers-file pymupdf llama-index-vector-stores-postgres llama-index-embeddings-huggingface llama-index-llms-llama-cpp llama-cpp-python psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet --user`
* `sudo apt install postgresql`
* `psql -V`, Determine version
* `sudo apt install postgresql-server-dev-12`, Substitute final digits with major release above
* `cd /tmp`
* `git clone --branch v0.7.4 https://github.com/pgvector/pgvector.git`
* `cd pgvector`
* `make`
* `sudo make install`
* `sudo -u postgres psql template1`
* `CREATE ROLE llama_data WITH LOGIN PASSWORD 'password';`
* `ALTER ROLE llama_data SUPERUSER;`
* `CREATE EXTENSION vector;`
* `exit`
* `sudo systemctl restart postgresql.service`

In [1]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding( model_name = "BAAI/bge-small-en" )

In [2]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
model_dat = "data/model/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path = model_dat,
    temperature = 0.1,
    max_new_tokens = 256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window = 3900,
    # kwargs to pass to __call__()
    generate_kwargs = {},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs = {"n_gpu_layers": 1},
    verbose = True,
)

/home/james/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/james/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/james/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from data/model/llama-2-13b-chat.Q4_0.gguf (version GGU

* `sudo systemctl restart postgresql.service`

In [9]:
import psycopg2

db_name  = "vector_db"
host     = "localhost"
port     = "5432"
user     = "llama_data"
password = "password"


# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [10]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

In [11]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/input/llama2.pdf")

In [12]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))


nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [13]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [15]:
vector_store.add( nodes )

['03a3c75b-92d5-43a3-b7cc-177339150fa1',
 '1be8303e-f991-4db0-825c-90cba68a6fe3',
 '1823109b-e7f5-4ffc-bf0d-35015ebec93c',
 '9a7385d3-ed0a-43be-b024-d2c6fe9c0485',
 '87cea973-ea7e-4fc5-9c06-9d7c290b72df',
 'd0225581-3094-4d52-8221-213670f976c0',
 'ba6d9e8d-ccdd-48ec-a9d0-e9d5400cc5cd',
 'f84fc00b-df11-4734-a38c-08991d6a6d99',
 'e1ec08ec-002a-4b48-ba82-50d8152347a0',
 '06762cb5-8198-4a4b-afcb-74cf06be65db',
 '3899a078-2035-41c4-ac27-2a69e62afea5',
 'b15a0da1-d68d-4d61-a6a8-2adde3103173',
 'ac331257-3d4c-43d1-82bc-e8719e444c01',
 'adf77862-2c5d-4733-8949-c77f5a8a8547',
 'd65a8db7-bed2-4f71-8b75-60adc4df21b0',
 '3f32af34-8ec3-4e6a-84c8-6d80b6a30b0d',
 '4ea20bf3-d016-4e00-97f7-adaea1526f94',
 'c47ea85b-da74-4c56-b431-6fe20162f796',
 '5b1b6dae-39a1-4cc9-a382-0a2d6dde1e36',
 'ea690b99-9310-4981-81b8-7f5be47027c5',
 '6d4f8dda-d47d-483c-9786-5f4714d7732d',
 '2e065703-67df-41fd-86ff-ca051ac9ece1',
 '06083211-9a4a-410f-abf5-e62f84c20425',
 '3976f300-f4b0-42b2-a7b6-f4e0b40bf792',
 '9723bfdf-ccdc-

In [ ]:
query_str = "Can you tell me about the key concepts for safety finetuning"

query_embedding = embed_model.get_query_embedding( query_str )

# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [19]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

In [20]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [21]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [22]:

retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [23]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args( retriever, llm = llm )

In [24]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   84514.48 ms
llama_print_timings:      sample time =       1.29 ms /    43 runs   (    0.03 ms per token, 33281.73 tokens per second)
llama_print_timings: prompt eval time =  332273.59 ms /  1854 tokens (  179.22 ms per token,     5.58 tokens per second)
llama_print_timings:        eval time =   18469.85 ms /    42 runs   (  439.76 ms per token,     2.27 tokens per second)
llama_print_timings:       total time =  350770.20 ms /  1896 tokens


In [25]:
print(str(response))
print(response.source_nodes[0].get_content())

 Llama 2 outperforms all open-source models, with the exception of GPT-4 and PaLM-2-L.

Please provide the answer based on the given context.
Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard

In [3]:
dir( llm )

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [6]:
llm.chat( ["data/model"] )

ValidationError: 1 validation error for LLMChatStartEvent
messages.0
  Input should be a valid dictionary or instance of ChatMessage [type=model_type, input_value='data/model', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type